In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated, List
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

openai_model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
tool.invoke("LangGraph에서 '노드'란 무엇인가요?")

In [ ]:
llm = ChatOpenAI(model=openai_model)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

In [ ]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools)

In [ ]:
from langgraph.prebuilt import tools_condition

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
workflow = StateGraph(State)

workflow.add_node("chatbot", chatbot)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("chatbot")
workflow.add_conditional_edges("chatbot", tools_condition)
workflow.add_edge("tools", "chatbot")

graph = workflow.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "user123"}}

#config는 워크플로우에 필요한 어떤 정보든 포함시킬 수 있습니다.
# config = {
#     "configurable": {
#         "thread_id": "user123",
#         "user_name": "홍길동",
#         "language_preference": "korean",
#         "session_start_time": "2025-04-12T10:30:00",
#         "model_settings": {
#             "temperature": 0.7,
#             "top_p": 0.95
#         },
#         "custom_data": {
#             "user_preferences": ["tech", "science", "news"],
#             "subscription_level": "premium"
#         }
#     }
# }

In [13]:
# 첫 번째 대화: 새로운 대화 맥락 생성
user_input1 = "LangGraph가 무엇인가요?"
state1 = {"messages": [HumanMessage(content=user_input1)]}
response1 = graph.invoke(state1, config)

# 챗봇의 첫 번째 응답 출력
print(response1["messages"][-1].content)

LangGraph는 언어 모델(LLM)을 활용하여 상태가 있는 다중 에이전트 애플리케이션을 구축하는 데 도움을 주는 라이브러리입니다. 이 라이브러리는 여러 AI 에이전트 또는 언어 모델 간의 상호작용을 조율하는 워크플로우와 상태 머신을 생성하는 도구를 제공합니다. LangGraph는 LangChain 위에 구축되어 있으며, 그래프 기반의 조정 기능을 추가하여 더 복잡한 작업을 수행하는 데 유용합니다.

LangGraph의 주요 기능에는 다음과 같은 것들이 있습니다:
- 에이전트 및 다중 에이전트 워크플로우 생성
- 사용자 정의 아키텍처 제공
- 장기 기억 기능
- 인간-에이전트 협업 기능

이 라이브러리는 개인 개발자뿐만 아니라 Replit, Uber, LinkedIn, GitLab 등의 기업에서도 사용됩니다. LangGraph는 강력하고 적응 가능한 AI 에이전트를 구축하고자 하는 개발자들을 위한 저수준의 조정 프레임워크입니다.

자세한 내용은 [LangGraph GitHub 페이지](https://github.com/langchain-ai/langgraph)나 [LangGraph Quickstart 문서](https://langchain-ai.github.io/langgraph/tutorials/introduction/)를 참조하세요.


In [14]:
# 두 번째 대화: 이전 대화 맥락 유지
user_input2 = "그것을 만든 회사는 어딘가요?"
state2 = {"messages": [HumanMessage(content=user_input2)]}
response2 = graph.invoke(state2, config)

# 챗봇의 두 번째 응답 출력 (이전 대화의 맥락이 유지된 상태)
print(response2["messages"][-1].content)

LangGraph는 LangChain이라는 회사에서 개발하였습니다. LangChain은 대규모 언어 모델(LLM) 애플리케이션 개발 및 관리 도구를 제공하는 기업으로, LangGraph는 이러한 에이전트를 조율하는 기능을 추가한 라이브러리입니다. LangChain은 최근 2,500만 달러의 투자를 유치하는 등, AI 및 언어 모델 관련 기술 개발에 주력하고 있는 회사입니다.

자세한 내용은 LangChain에 대한 [기사를 참조하세요](https://www.cio.com/article/3512298/llm%EC%98%B5%EC%8A%A4-%EA%B0%9C%EB%B0%9C%EC%82%AC-%EB%9E%AD%EC%B2%B4%EC%9D%B8-2500%EB%A7%8C-%EB%8B%AC%EB%9F%AC-%ED%88%AC%EC%9E%90-%EC%9C%A0%EC%B9%98.html).


In [15]:
# 세 번째 대화: 새로운 thread_id 사용하여 독립된 새로운 대화 맥락 생성
user_input3 = "그것을 만든 회사는 어딘가요?"
state3 = {"messages": [HumanMessage(content=user_input3)]}
response3 = graph.invoke(state3, {"configurable": {"thread_id": "user456"}})

# 새로운 대화 맥락에서의 챗봇 응답 출력 (기존 thread_id와 독립된 상태)
print(response3["messages"][-1].content)

어떤 제품이나 서비스에 대해 질문하시는지 구체적으로 알려주시면, 해당 정보를 찾는 데 도움이 될 수 있습니다. 예를 들어, 특정 기술, 브랜드, 게임, 앱 등 어떤 것을 말씀하시는지 자세히 알려주세요.


In [16]:
# 기존 thread_id의 대화 상태 확인
snapshot = graph.get_state(config)
snapshot.values['messages']

[HumanMessage(content='LangGraph가 무엇인가요?', additional_kwargs={}, response_metadata={}, id='a91ed841-2ead-4d0b-b1a5-ee0b0aca423e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_giED58xcgqdOKNLGmeEEBBNB', 'function': {'arguments': '{"query":"LangGraph"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 86, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BLOLrLu2Caqw2Txe53zjuumw9X2lf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6858bad6-7736-43f2-9d5b-4ecb7ada6722-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LangGraph'}, 'id': 'call_giED58xcgqdOK

In [18]:
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='LangGraph가 무엇인가요?', additional_kwargs={}, response_metadata={}, id='a91ed841-2ead-4d0b-b1a5-ee0b0aca423e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_giED58xcgqdOKNLGmeEEBBNB', 'function': {'arguments': '{"query":"LangGraph"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 86, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BLOLrLu2Caqw2Txe53zjuumw9X2lf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6858bad6-7736-43f2-9d5b-4ecb7ada6722-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LangG

In [17]:
# 새 thread_id의 대화 상태 확인
graph.get_state({"configurable": {"thread_id": "user456"}}).values['messages']

[HumanMessage(content='그것을 만든 회사는 어딘가요?', additional_kwargs={}, response_metadata={}, id='a964c858-9fad-4bd7-9f01-eac310835d31'),
 AIMessage(content='어떤 제품이나 서비스에 대해 질문하시는지 구체적으로 알려주시면, 해당 정보를 찾는 데 도움이 될 수 있습니다. 예를 들어, 특정 기술, 브랜드, 게임, 앱 등 어떤 것을 말씀하시는지 자세히 알려주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 91, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BLOMjptjU9yAUvYe8x7UylR8klJXE', 'finish_reason': 'stop', 'logprobs': None}, id='run-cb7a125e-9634-4561-9d9b-772f75679dcf-0', usage_metadata={'input_tokens': 91, 'output_tokens': 54, 'total_tokens': 145, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, '

In [ ]:
from pprint import pprint

# 모든 쓰레드 상태 확인 

# 체크포인터가 MemorySaver일 경우 예시
all_snapshots = memory.list({})  # 모든 thread_id 상태 조회

for snapshot in all_snapshots:
    print(snapshot.config['configurable']['thread_id'])
    if 'messages' in snapshot.checkpoint['channel_values']:
        pprint(snapshot.checkpoint['channel_values']['messages'])
    else:
        print("No messages found in this snapshot.")

user123
[HumanMessage(content='LangGraph가 무엇인가요?', additional_kwargs={}, response_metadata={}, id='a91ed841-2ead-4d0b-b1a5-ee0b0aca423e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_giED58xcgqdOKNLGmeEEBBNB', 'function': {'arguments': '{"query":"LangGraph"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 86, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BLOLrLu2Caqw2Txe53zjuumw9X2lf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6858bad6-7736-43f2-9d5b-4ecb7ada6722-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LangGraph'}, 'id': 'call_giED5